# Анализ работы модели предсказания качества воздуха

В работе используются:
- модель с [Kaggle](https://www.kaggle.com/code/hamedetezadi/air-quality-prediction), в формате [ноутбука](https://colab.research.google.com/drive/14Tlqrmg76i7S33Gc5P5iFvjh7Ml2dXxB?usp=sharing)
- датасет  «СОСТОЯНИЕ ЗАГРЯЗНЕНИЯ АТМОСФЕРЫ В
 ГОРОДАХ РОССИИ C 2007 ГОДА» - приложено в файлах ноута


# Предварительная обработка данных для анализа качества воздуха


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shrutibhargava94/india-air-quality-data")

print("Path to dataset files:", path)

100%|██████████| 5.22M/5.22M [00:00<00:00, 74.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shrutibhargava94/india-air-quality-data/versions/1


## Запускаем модель

In [ ]:
import joblib
# Загрузка обученной модели
model_path = 'Air_Quality_Prediction.joblib'  # Укажите путь к вашей модели
try:
    model = joblib.load(model_path)
    print("Модель успешно загружена.")
except FileNotFoundError:
    print(f"Ошибка: модель не найдена по пути {model_path}")
    model = None

# Применение модели для прогнозов (если модель загружена)
if model:
    try:
        predictions = model.predict(data_encoded)
        data_encoded['Predictions'] = predictions
        print("Прогнозы успешно выполнены.")

        # Сохранение данных с прогнозами
        predictions_file_path = 'predictions_air_quality.csv'
        data_encoded.to_csv(predictions_file_path, index=False)
        print(f"Данные с прогнозами сохранены в файл {predictions_file_path}")
    except Exception as e:
        print(f"Ошибка при выполнении прогнозов: {e}")

Модель успешно загружена.
Ошибка при выполнении прогнозов: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- air_co_emissions
- air_general_level_1
- air_general_level_2
- air_general_level_3
- air_general_level_4
- ...



In [ ]:
expected_features = model.feature_names_in_
print("Ожидаемые моделью признаки:", expected_features)
print("Текущие признаки данных:", data_encoded.columns.tolist())


Ожидаемые моделью признаки: ['SOi' 'Noi' 'Rpi' 'SPMi']
Текущие признаки данных: ['year', 'region_oktmo', 'city_oktmo', 'air_repeatability', 'air_qcp', 'air_solid_emissions', 'air_so_emissions', 'air_no_emissions', 'air_co_emissions', 'air_population', 'air_stantions', 'region_Амурская область', 'region_Архангельская область', 'region_Астраханская область', 'region_Белгородская область', 'region_Брянская область', 'region_Владимирская область', 'region_Волгоградская область', 'region_Вологодская область', 'region_Воронежская область', 'region_Еврейская автономная область', 'region_Забайкальский край', 'region_Ивановская область', 'region_Иркутская область', 'region_Калининградская область', 'region_Калужская область', 'region_Камчатский край', 'region_Карачаево-Черкесская Республика', 'region_Кемеровская область', 'region_Кировская область', 'region_Костромская область', 'region_Краснодарский край', 'region_Красноярский край', 'region_Курганская область', 'region_Курская область', 'regi

In [ ]:
import pandas as pd
import numpy as np
import joblib
import re

# Загрузка данных
file_path = 'data_air_cities_100_v20231129.csv'
data = pd.read_csv(file_path, sep=';', engine='python')

# Очистка числовых данных от текста
def extract_first_number(value):
    if isinstance(value, str):
        match = re.search(r'\d+(\.\d+)?', value)
        return float(match.group()) if match else np.nan
    return value

numeric_columns = ['air_solid_emissions', 'air_so_emissions', 'air_no_emissions', 'air_co_emissions', 'air_population', 'air_repeatability']
for col in numeric_columns:
    data[col] = data[col].apply(extract_first_number)

# Обработка пропущенных значений
categorical_columns = ['region', 'city', 'air_general_level', 'air_standard_index']

# Заполнение числовых данных средним значением
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

# Заполнение категориальных данных модой
for col in categorical_columns:
    data[col] = data[col].fillna(data[col].mode()[0])

# Нормализация числовых данных (мин-макс нормализация)
data[numeric_columns] = (data[numeric_columns] - data[numeric_columns].min()) / (
    data[numeric_columns].max() - data[numeric_columns].min()
)

# Кодирование категориальных данных
data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Загрузка модели
model_path = 'Air_Quality_Prediction.joblib'
model = joblib.load(model_path)

# Подготовка данных для модели
expected_features = model.feature_names_in_
missing_features = set(expected_features) - set(data_encoded.columns)

# Добавляем отсутствующие признаки, заполняя их нулями
for feature in missing_features:
    data_encoded[feature] = 0

# Оставляем только нужные признаки для модели
data_final = data_encoded[list(expected_features)]

# Применение модели для прогнозов
predictions = model.predict(data_final)
data_final['Predictions'] = predictions

# Сохранение обработанных данных
processed_file_path = 'processed_data_air_quality.csv'
data_final.to_csv(processed_file_path, index=False)
print(f"Обработанные данные сохранены в {processed_file_path}")

# Сохранение прогнозов
predictions_file_path = 'predictions_air_quality.csv'
data_final.to_csv(predictions_file_path, index=False)
print(f"Прогнозы сохранены в {predictions_file_path}")


Обработанные данные сохранены в processed_data_air_quality.csv
Прогнозы сохранены в predictions_air_quality.csv


<ipython-input-13-5b05d44382f1>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['Predictions'] = predictions
